In [5]:
#Logistic regression 
#Feature : Bag of work for attendency of event, average time/slope age, average date of competition, age


#this scipt is used for creating the data base
import csv
from enum import Enum

#type of event
class EventT(Enum):
    marathon = 1
    halfMarathon = 2
    fiveKm = 3
    tenKm = 4
    fifthteen = 5
    twenty5 = 6
    thirty = 7
    cycling = 8
    multiAthlon = 9
    ironman = 10
    ultraTrail = 11
    others = 0
    
    
outfile = open ('dataPrediction.csv','w')
writer = csv.writer(outfile)

with open ('Data/Project1_data.csv') as csvfile:
    
    reader = csv.reader(csvfile)
    for i, row in enumerate(reader):
        
        wrRow = []
        numDate = 0
        
        if(i != 0):
            for j, col in enumerate(row):
                
                if j == 0:
                    wrRow.append(i-1)
                    
                elif (j-1)%5 == 0: #date
                    yy,mm,dd = col.split("-")
                    numDate = (int(yy)-2012)* 12 + int(mm)-12
                    if numDate > 0:
                        wrRow.append(numDate)
                if numDate > 0:
                    if (j-1)%5 == 1: #event name 
                        eventName = col.lower()
                        eventName = eventName.replace(',','.')

                    elif (j-1)%5 == 2: #event type

                        eventType = col.lower()
                        eventType = eventType.replace(',','.')

                        if 'athlon' in eventType or 'athlon' in eventName or '5i50' in eventType:
                            typeOfEvent = EventT.multiAthlon.value

                        elif 'cycl' in eventType or 'cycl' in eventName or 'velo' in eventType or 'velo' in eventName:
                            typeOfEvent = EventT.cycling.value

                        elif 'ironman' in eventType or 'ironman' in eventName:
                            typeOfEvent = EventT.ironman.value

                        elif 'ultra-trail' in eventName or 'harricana' in eventName:
                            typeOfEvent = EventT.ultraTrail.value

                        #range of 4 to 6 km
                        elif '5 km' in eventType or '5k' in eventType or '5.5 km' in eventType or '6 km' in eventType or '6k' in eventType or '4 km' in eventType:
                            typeOfEvent = EventT.fiveKm.value

                        #range 8 to 11km
                        elif '10 km' in eventType or '9 km' in eventType or '8 km' in eventType or '10km' in eventType or '11 km' in eventType or '10k' in eventType:
                            typeOfEvent = EventT.tenKm.value

                        #range 12 to 15 km
                        elif '12 km' in eventType or '13 km' in eventType or '15 km' in eventType:
                            typeOfEvent = EventT.fifthteen.value

                        #range of 23 to 25 km
                        elif '23 km' in eventType or '23.5 km' in eventType or '25 km' in eventType or '25km' in eventType:
                            typeOfEvent = EventT.twenty5.value

                        #range of 28 to 33 km
                        elif '28 km' in eventType or '30 km' in eventType or '33 km' in eventType:
                            typeOfEvent = EventT.thirty.value    

                        #range of 18 to 22.2 km
                        elif 'demi' in eventType or 'demi' in eventName or 'half' in eventType or 'half' in eventName or '18 km' in eventType or '19 km' in eventType or '20 km' in eventType or '21 km' in eventType or '21.1 km' in eventType or '21km' in eventType or '22 km' in eventType or '22.2 km' in eventType:
                            typeOfEvent = EventT.halfMarathon.value

                        #range of 42 to 50 km
                        elif ('marathon' in eventName or 'marathon' in eventType  or '42.2 km' in eventType or '45 km' in eventType or '48 km' in eventType or '50 km' in eventType) and '1 km' not in eventType:
                            typeOfEvent = EventT.marathon.value

                        else:
                            typeOfEvent = EventT.others.value

                        wrRow.append(typeOfEvent)


                    elif (j-1)%5 == 3: #event time
                        if col != '-1':
                            hh,mi,se = col.split(":")
                            time = int(hh)*60+ int(mi)
                            wrRow.append(time)
                        else:
                            wrRow.append(col)

                    elif (j-1)%5 == 4: #category
                        ageS = col.lower()

                        if 'no age' in ageS or 'male' in ageS or 'female' in ageS:
                            age=-1
                            wrRow.append(age)
                        else:
                            try:
                                minAge = int(ageS[1:3])
                            except ValueError:
                                minAge = -1;
                            try:
                                maxAge = int(ageS[4:6])
                            except ValueError:
                                maxAge = minAge

                            age = (maxAge+minAge)/2

                            wrRow.append(age)
                         

            writer.writerow(wrRow)
                


In [6]:
#Logistic regression 
#Feature : Bag of work for attendency of event, average time/slope age, average date of competition, age


#this scipt is used to normalize the result of runners for 10k to 30k run
import csv
import numpy
from enum import Enum

#type of event
class EventT(Enum):
    marathon = 1
    halfMarathon = 2
    fiveKm = 3
    tenKm = 4
    fifthteen = 5
    twenty5 = 6
    thirty = 7
    cycling = 8
    multiAthlon = 9
    ironman = 10
    ultraTrail = 11
    others = 0

with open ('dataPrediction.csv') as csvfile:
    
    sum5k=0
    sum10k=0
    sum15k=0
    sum22k=0
    sum25k=0
    sum30k=0
    sum42k=0
    itt5k=0
    itt10k=0
    itt15k=0
    itt22k=0
    itt25k=0
    itt30k=0
    itt42k=0
    n=0
    
    reader = csv.reader(csvfile)
    for i, row in enumerate(reader):
        
        n=i
        wrRow = [] 
        avg5k=0
        avg10k=0
        avg15k=0
        avg22k=0 # half marathon
        avg25k=0
        avg30k=0
        avg42k=0 #full marathon
        
        it5k=0
        it10k=0
        it15k=0
        it22k=0
        it25k=0
        it30k=0
        it42k=0
        
        evType=0
        
        for j, col in enumerate(row):
            
            
            
            if j == 0:
                evType=0
            #elif (j-1)%4 == 0 or (j-1)%4 == 3: #dont care about age and date of event
                
            elif (j-1)%4 == 1:
                evType = numpy.int(col)
            
            elif (j-1)%4 == 2 and numpy.int(col) != -1:
            
                if evType == EventT.fiveKm.value:
                    it5k += 1
                    avg5k += numpy.int(col)
                    
                elif evType == EventT.tenKm.value:
                    it10k += 1
                    avg10k += numpy.int(col)
                    
                elif evType == EventT.fifthteen.value:
                    it15k += 1
                    avg15k += numpy.int(col)
                    
                elif evType == EventT.halfMarathon.value:
                    it22k += 1 
                    avg22k += numpy.int(col)
                    
                elif evType == EventT.twenty5.value:
                    it25k += 1 
                    avg25k += numpy.int(col)
                    
                elif evType == EventT.thirty.value:
                    it30k += 1
                    avg30k += numpy.int(col)
                    
                elif evType == EventT.marathon.value:
                    it42k += 1
                    avg42k += numpy.int(col)
                    
        if it42k != 0: 
            
            itt42k += 1
            avg42k /= it42k
            sum42k+=avg42k
            
            if it5k!=0:
                itt5k += 1
                sum5k+=avg42k/(avg5k/it5k)

            if it10k!=0:
                itt10k += 1
                sum10k+=avg42k/(avg10k/it10k)

            if it15k!=0:
                itt15k += 1
                sum15k+=avg42k/(avg15k/it15k)

            if it22k!=0:
                itt22k += 1
                sum22k+=avg42k/(avg22k/it22k)

            if it25k!=0:
                itt25k += 1
                sum25k+=avg42k/(avg25k/it25k)
                
            if it30k!=0:
                itt30k += 1
                sum30k+=avg42k/(avg30k/it30k)

            
            

            
sum5k = sum5k/itt5k
sum10k = sum10k/itt10k
sum15k = sum15k/itt15k
sum22k = sum22k/itt22k
sum25k = sum25k/itt25k
sum30k = sum30k/itt30k
sum42k = sum42k/itt42k

print("ratio 42k/30k = " , sum30k)
print("ratio 42k/25k = " , sum25k)
print("ratio 42k/22k = " , sum22k)
print("ratio 42k/15k = " , sum15k)
print("ratio 42k/10k = " , sum10k)
print("ratio 42k/5k = " , sum5k)
                

ratio 42k/30k =  1.5567568263983844
ratio 42k/25k =  2.0681804134822044
ratio 42k/22k =  2.3294321306940513
ratio 42k/15k =  3.440985946719747
ratio 42k/10k =  5.048230333352175
ratio 42k/5k =  8.417810771613237


In [8]:
#Logistic regression 
#Feature : Bag of work for attendency of event, average time/slope age, average date of competition, age

#this scipt is used for creating the feature vector
import csv
import numpy
from enum import Enum

ratio30k = sum30k
ratio25k = sum25k
ratio22k = sum22k
ratio15k = sum15k
ratio10k = sum10k
ratio5k = sum5k


#type of event
class EventT(Enum):
    marathon = 1
    halfMarathon = 2
    fiveKm = 3
    tenKm = 4
    fifthteen = 5
    twenty5 = 6
    thirty = 7
    cycling = 8
    multiAthlon = 9
    ironman = 10
    ultraTrail = 11
    others = 0
    avgTime = 12
    slope = 13
    avgDate = 14
    age = 15
    bias = 16
    
    
outfile = open ('vectorPrediction.csv','w')
writer = csv.writer(outfile)


with open ('dataPrediction.csv') as csvfile:
    
    reader = csv.reader(csvfile)
    for i, row in enumerate(reader):
        
        featureVec = [0] * 17
        
        averageDate = 0
        averageTime = 0
        slope = 0
        matrixX = []
        matrixY = []
        date=0
        avgAge=[]
        eType = 0
        avgerageDate = 0
        attended2015=0
        featureVec[EventT.bias.value] = 1
        
        for j, col in enumerate(row):
        
            if j == 0:
                n=0
            
            elif (j-1)%4 == 0: # date
                date = numpy.int(col)
                avgerageDate += date
                n = n+1
                
            elif (j-1)%4 == 1: #event type
                eType = numpy.int(col)
                           
                if eType == EventT.marathon.value and date == 45:
                    attended2015 = 1
                    avgerageDate-=date
                    n=n-1
                else:
                    featureVec[eType] = featureVec[eType]+1
                
            elif (j-1)%4 == 2 and numpy.int(col) != -1 and not (eType == EventT.marathon.value and date == 45): #time
                if eType == EventT.fiveKm.value:
                    matrixX.append(date)
                    matrixY.append(numpy.int(col)*ratio5k)
                    
                if eType == EventT.tenKm.value:
                    matrixX.append(date)
                    matrixY.append(numpy.int(col)*ratio10k)
                    
                elif eType == EventT.fifthteen.value:
                    matrixX.append(date)
                    matrixY.append(numpy.int(col)*ratio15k)
                    
                elif eType == EventT.halfMarathon.value:
                    matrixX.append(date)
                    matrixY.append(numpy.int(col)*ratio22k)
                    
                elif eType == EventT.twenty5.value:
                    matrixX.append(date)
                    matrixY.append(numpy.int(col)*ratio25k)
                    
                elif eType == EventT.thirty.value:
                    matrixX.append(date)
                    matrixY.append(numpy.int(col)*ratio30k)
                    
                elif eType == EventT.marathon.value:
                    matrixX.append(date)
                    matrixY.append(numpy.int(col))
                    
               
            elif (j-1)%4 == 3 and not (eType == EventT.marathon.value and date == 45) and numpy.float(col) != -1: #age
                avgAge.append(numpy.float(col)/4)
        
        if n > 0:  #if the partitipant did somthing else than juste the 2015 marathon    
            if len(avgAge) > 0:
                featureVec[EventT.age.value] = sum(avgAge)/(len(avgAge))
            featureVec[EventT.avgDate.value] = avgerageDate/(n*4)
            x = numpy.zeros((len(matrixX),2))
            y = numpy.zeros((len(matrixX),1))

            for n, val in enumerate(matrixX):
                x[n,0] = val
                x[n,1] = 1
                y[n,0] = matrixY[n]



            if len(matrixX) > 1:            
                w = numpy.array([[0],[y[0]]])
                try:
                    w = numpy.linalg.inv(numpy.dot(x.transpose(1,0) , x))
                    w = numpy.dot(w,numpy.dot(x.transpose(1,0),y))
                except:
                    mean = numpy.mean(y,axis=0)                
                    w = numpy.array([[0],[mean[0]]])

            elif len(matrixX) > 0:

                w = numpy.array([[0],[y[0,0]]])
            else :
                w = numpy.array([[0],[0]])


            featureVec[EventT.slope.value] = w[0,0]
            if len(matrixY) > 0:
                featureVec[EventT.avgTime.value] = sum(matrixY)/(len(matrixY)*125)
        else:
            attended2015 = -1
        
        writer.writerow(featureVec)


In [28]:
#Logistic regression 
#Feature : Bag of work for attendency of event, average time/slope age, average date of competition, age

#this scipt used the feature vector and build a modal (w)
import random

vectorCSV = numpy.loadtxt('vectorPrediction.csv',dtype=numpy.float, delimiter=',')
w = numpy.loadtxt("w.out")

outfile = open ('prediction.csv','w') # the vector for the actual prediction of 2016
writerPrediction = csv.writer(outfile)
a=[]
for i, float in enumerate(vectorCSV):
    tv = numpy.array([[value] for value in vectorCSV[i]])

    exp3 = -1*numpy.dot(w,tv)
    if exp3 < 100:            
        exp2 = 1/(1+numpy.exp(exp3))
    else:
        exp2 = 0
    result = []
    result.append(numpy.squeeze(numpy.round((exp2))))
    writerPrediction.writerow(result)
    a.append(numpy.squeeze(numpy.round((exp2))))


In [29]:
print(len(a))

8711


In [25]:
print(w)

[  -46.13144246 -3913.67324638 -1028.2063609   -307.02485669  -123.49901191
     7.36271143  -269.8641503      9.14825502  -419.68629779   -76.76007962
 -1422.81506176  -309.08290988  -930.88838774   133.39089625  1002.38293097
   -40.57112054  -128.76707606]
